In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplot plt

import pyspark
from pyspark.sql import SparkSession

#spark = SparkSession.builder.appName("PySpark").getOrCreate()
#spark
spark = SparkSession.builder \
    .appName("PySpark") \
    .getOrCreate()


Calculation started (calculation_id=7ec3e864-1ce4-9a89-7a2f-a8dc9a1f7578) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation 7ec3e864-1ce4-9a89-7a2f-a8dc9a1f7578 failed


  File "<stdin>", line 7, in <module>
AttributeError: 'function' object has no attribute 'appName'



In [2]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

Calculation started (calculation_id=68c3e864-235b-4a6f-f4d2-e6a384f72f3a) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [3]:
def printdf(df, l=5):
    return df.limit(l).toPandas()

def nullcount(df):
    return {col: df.filter(df[col].isNull()).count() for col in df.columns}

def shape(df):
    # df.toPandas().shape
    print((df.count(), len(df.columns)))

Calculation started (calculation_id=c8c3e864-28e6-a710-94f0-4dee97b42502) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [4]:
data = spark.read.csv(f"s3://evo-prod-cdc-copy-query/pipeline-invoice/0a803ee2-20fc-49b5-aca9-ce270a18644f.csv", 
                      inferSchema=True, 
                      header=True)
data.printSchema()

Calculation started (calculation_id=a0c3e864-2a42-2482-256a-3b8b8052d482) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
root
 |-- creation_date: string (nullable = true)
 |-- created_date_time: long (nullable = true)
 |-- sg_time: string (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- invoice_number: string (nullable = true)
 |-- total_amount: double (nullable = true)



# EDA 

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

Calculation started (calculation_id=58c3e864-405b-b3e3-b982-9e647198bca7) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [6]:
def printdf(df, l=5):
    return df.limit(l).toPandas()

def nullcount(df):
    return {col: df.filter(df[col].isNull()).count() for col in df.columns}

def shape(df):
    # df.toPandas().shape
    print((df.count(), len(df.columns)))

Calculation started (calculation_id=74c3e864-45ab-aa7a-9c16-db958b50741f) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [7]:
print('Dataframe dimensions:', (data.count(), len(data.columns)))
printdf(data)

Calculation started (calculation_id=c6c3e864-46f2-fc43-8fdf-b741cc914389) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
Dataframe dimensions: (431, 6)
         creation_date  created_date_time  ... invoice_number  total_amount
0  2023-03-01 12:29:46   1677644986326689  ...        AMI0104        2000.0
1  2023-02-28 01:11:06   1677561066343199  ...         NUR109        1500.0
2  2023-02-28 01:09:32   1677560972531807  ...          CM103         750.0
3  2023-02-27 08:13:18   1677499998057097  ...        MOH0131         250.0
4  2023-02-27 09:47:32   1677462452627054  ...           JW05        3000.0

[5 rows x 6 columns]



In [8]:
data.toPandas().describe()

Calculation started (calculation_id=cac3e864-5874-8a79-7a3f-f18f6b9d7a5a) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
       created_date_time     user_id  total_amount
count       4.310000e+02  431.000000    431.000000
mean        1.632474e+15  307.793503   1085.243794
std         1.447563e+14  194.630948   3873.099011
min         9.783072e+14    3.000000      0.000000
25%         1.657736e+15  144.000000    200.000000
50%         1.664550e+15  324.000000    450.000000
75%         1.672065e+15  517.000000    951.000000
max         1.677645e+15  597.000000  75000.000000



In [9]:
rtl_data = data.filter(F.col("user_id").isNotNull())

rtl_data.toPandas().shape

printdf(rtl_data)

Calculation started (calculation_id=fcc3e864-5dec-6614-5697-3a4df89c8431) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
         creation_date  created_date_time  ... invoice_number  total_amount
0  2023-03-01 12:29:46   1677644986326689  ...        AMI0104        2000.0
1  2023-02-28 01:11:06   1677561066343199  ...         NUR109        1500.0
2  2023-02-28 01:09:32   1677560972531807  ...          CM103         750.0
3  2023-02-27 08:13:18   1677499998057097  ...        MOH0131         250.0
4  2023-02-27 09:47:32   1677462452627054  ...           JW05        3000.0

[5 rows x 6 columns]



# RFM Modeling

In [10]:
df = rtl_data.toPandas()

print(df.creation_date.min())
print(df.creation_date.max())

Calculation started (calculation_id=90c3e864-6f6d-2f1c-5430-9e1f2c89ccdd) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
2001-01-01 08:00:00
2023-03-01 12:29:46



In [11]:
#max_date_row = rtl_data.select(F.max('creation_date')).collect()[0]
#max_date_str = max_date_row['max(creation_date)']
#print(max_date_str)
import datetime

now = datetime.datetime.now()

Calculation started (calculation_id=86c3e864-74c2-e8f5-a6d6-53c48ff308e7) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [12]:
# Recency   = Overall latest invoice date - individual customer's last invoice date
# Frequency = count of invoice no. of transaction(s)
# Monetary = Sum of Total amount for each customer

# Set max_date_str as the overall last invoice date. This is to calculate recency in days.
latest_date = F.to_date(F.lit(now), 'yyyy-MM-dd HH:mm:ss')

# Create RFM Modelling scores for each customer

### Approach One
rfm_scores = (rtl_data.groupBy("user_id")
              .agg((F.datediff(latest_date, F.max(F.col("creation_date")))).alias("Recency"),
                   F.count("*").alias("Frequency"),
                   F.sum(F.col("total_amount")).alias("Monetary")).sort("user_id"))

### Approach Two
# recency = (
#     rtl_data.groupBy("user_id")
#             .agg((F.datediff(latest_date, F.max(F.col("creation_date")))).alias("Recency"))
# )

# frequency = (
#     rtl_data.groupBy("user_id", "invoice_number").count()
#             .groupBy("user_id")
#             .agg(F.count("*").alias("Frequency"))
# )

# monetary = (
#     rtl_data.groupBy("user_id")
#             .agg(F.sum(F.col("total_amount")).alias("Monetary"))
# )

# rfm_scores = (
#     recency.join(frequency, 'user_id', how = 'inner')
#            .join(monetary,'user_id', how = 'inner')
# )

printdf(rfm_scores)

Calculation started (calculation_id=f0c3e864-7614-fe28-ecb5-16311a256996) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
   user_id  Recency  Frequency  Monetary
0        3     8154          3     361.0
1        5      181         17   11695.0
2       12       72         14    3524.0
3       15     8154          4   20000.0
4       24      170          8   46300.0



In [13]:
rfm_scores.toPandas().describe()
rfm_scores = rfm_scores.filter(rfm_scores.Recency < 500)

Calculation started (calculation_id=2ec3e864-879c-3dee-2d02-44584806d615) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


### Explore the RFM Scores using SNS

In [14]:
rfm_scores_df = rfm_scores.toPandas()

Calculation started (calculation_id=00c3e864-8d1e-4090-7f7a-135399d60cc6) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [15]:
import seaborn as sns

fig, ax = plt.subplots(1, 3, figsize=(16, 8))

# Recency distribution plot
sns.histplot(rfm_scores_df.query('Recency < 500')['Recency'], kde=True, ax=ax[0])

# Frequency distribution plot
sns.histplot(rfm_scores_df.query('Frequency < 35')['Frequency'], kde=True, ax=ax[1])

# Monetary distribution plot
sns.histplot(rfm_scores_df.query('Monetary < 100000')['Monetary'], kde=True, ax=ax[2])

Calculation started (calculation_id=7ac3e864-a2d7-a382-0483-be1f2fbe5e11) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<AxesSubplot:xlabel='Monetary', ylabel='Count'>



In [16]:
import boto3
s3client=boto3.client('s3')
fig.savefig('rfm_histogram.png')
s3client.upload_file('rfm_histogram.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_histogram.png')

Calculation started (calculation_id=10c3e864-a88f-7d39-02c3-252156501185) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


### Save RFM Numbers

In [17]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('athenapyspark-rfm-analysis-result')
import tempfile
with tempfile.TemporaryFile(mode='w+t') as f:
    # write the DataFrame to the temporary file
    rfm_scores_df.to_csv(f, index=False)
    # reset the file pointer to the beginning of the file
    f.seek(0)
    # upload the file to S3
    bucket.put_object(Key='analysis-results/evo_rfm_numbers.csv', Body=f.read().encode())

Calculation started (calculation_id=f6c3e864-ade8-b275-1433-b283ca08b744) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
0
s3.Object(bucket_name='athenapyspark-rfm-analysis-result', key='analysis-results/evo_rfm_numbers.csv')



### Calculate RFM Scores

In [18]:
# We are going to divide our customer base into 3 segments: 
#    1st segment: top 33%
#    2nd segment: 33% - 66%
#    3rd segment: bottom 33%

# Understand and calculate different quantiles

# rfm_scores.toPandas()["Recency"].describe()

segments = [0.33, 0.66]

quantiles = rfm_scores.approxQuantile(
    ["Recency", "Frequency", "Monetary"], 
    segments,
    0)
quantiles

Calculation started (calculation_id=56c3e864-b34c-c47a-7550-f4438b976925) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
[[132.0, 257.0], [2.0, 4.0], [700.0, 3600.0]]



In [19]:
# Calculate R,F,M scores considering 3 quantiles

rfm_scores = (rfm_scores
    .withColumn("R_Score",
                F.when(F.col("Recency") < quantiles[0][0], F.lit(1))
                 .when(F.col("Recency") < quantiles[0][1], F.lit(2))
                 .otherwise(F.lit(3)))
    .withColumn("F_Score",
                F.when(F.col("Frequency") < quantiles[1][0], F.lit(3))
                 .when(F.col("Frequency") < quantiles[1][1], F.lit(2))
                 .otherwise(F.lit(1)))
    .withColumn("M_Score",
                F.when(F.col("Monetary") < quantiles[2][0], F.lit(3))
                 .when(F.col("Monetary") < quantiles[2][1], F.lit(2))
                 .otherwise(F.lit(1))))

# printdf(rfm_scores.filter(F.col("CustomerID").isin([12346,12747,12748,12749,12820])))
# printdf(rfm_scores)

# rfm_scores.describe().show()

Calculation started (calculation_id=6ec3e864-cce7-c2de-1ed4-db47bce77df9) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [20]:
# Aggregate RFM scores

rfm_agg_scores = (
    rfm_scores
    .withColumn("RFM_Score", F.col("R_Score") + F.col("F_Score") + F.col("M_Score"))
    .withColumn("RFM_ScoreGroup", F.concat(F.col("R_Score").cast(StringType()), F.col("F_Score").cast(StringType()), F.col("M_Score").cast(StringType())))
)

printdf(rfm_agg_scores)

Calculation started (calculation_id=3ec3e864-d298-f607-296d-a0688efdaf64) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
   user_id  Recency  Frequency  ...  M_Score  RFM_Score  RFM_ScoreGroup
0        5      181         17  ...        1          4             211
1       12       72         14  ...        2          4             112
2       24      170          8  ...        1          4             211
3       31      192          6  ...        2          5             212
4       34      234         16  ...        2          5             212

[5 rows x 9 columns]



In [21]:
df_rfm_agg_scores = rfm_agg_scores.toPandas()
df_rfm_agg_scores.shape

Calculation started (calculation_id=f4c3e864-e431-f5ea-2196-06f89bbddabc) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
(82, 9)



In [22]:
df_rfm_agg_scores = df_rfm_agg_scores.sort_values(["RFM_Score"])

fig, ax = plt.subplots(1, 2, figsize=(16, 8))
plt.xticks(rotation=90)
# sns.histplot(df_rfm_agg_scores["RFMScore"], kde=True, bins=10, ax=ax[0])
sns.countplot(x="RFM_Score", data=df_rfm_agg_scores, 
              color='steelblue', ax=ax[0])
sns.countplot(x="RFM_ScoreGroup", data=df_rfm_agg_scores, 
              color='steelblue', ax=ax[1])
# sns.histplot(df_rfm_agg_scores["RFMScoreGroup"], color='steelblue', ax=ax[2])
fig.savefig('rfmscore_countplot.png')
s3client.upload_file('rfmscore_countplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfmscore_countplot.png')

Calculation started (calculation_id=32c3e864-e936-d8b9-e9be-73db915f1244) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Segmentation 1

In [23]:
# Identify the three segments again based on RFMScore (33% in each segment)
segments = [0.33, 0.66]
quantiles = rfm_agg_scores.approxQuantile("RFM_Score", segments, 0)
quantiles

Calculation started (calculation_id=24c3e864-ee87-0fb3-9fb0-953386b37749) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
[5.0, 7.0]



In [24]:
# Assign loyalty level to each customer
loyalty_level = ['Platinum', 'Gold', 'Silver']

rfm_loyalty = (rfm_agg_scores
    .withColumn("Loyalty",
                F.when((F.col("RFM_Score") <= quantiles[0]), F.lit(loyalty_level[0]))
                 .when((F.col("RFM_Score") <= quantiles[1]), F.lit(loyalty_level[1]))
                 .otherwise(F.lit(loyalty_level[2]))))

Calculation started (calculation_id=8cc3e865-082e-9079-3351-e2e8f32acdc2) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [25]:
printdf(rfm_loyalty)

Calculation started (calculation_id=80c3e865-0d8a-d7ab-1866-8b17839fc19c) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
   user_id  Recency  Frequency  ...  RFM_Score  RFM_ScoreGroup   Loyalty
0        5      181         17  ...          4             211  Platinum
1       12       72         14  ...          4             112  Platinum
2       24      170          8  ...          4             211  Platinum
3       31      192          6  ...          5             212  Platinum
4       34      234         16  ...          5             212  Platinum

[5 rows x 10 columns]



# inspect our loyalty segmentation

In [26]:
# inspect our loyalty segmentation

df_rfm_loyalty = rfm_loyalty.toPandas().sort_values(["RFM_ScoreGroup"])

fig, ax = plt.subplots(1, 1, figsize=(6, 8))

sns.countplot(x="Loyalty", data=df_rfm_loyalty, color='steelblue')
for p in ax.patches:
    ax.annotate('{}'.format(p.get_height()), (p.get_x() + 0.3, p.get_height() + 1))

fig.savefig('rfm_loyalty_countplot.png')
s3client.upload_file('rfm_loyalty_countplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_loyalty_countplot.png')

Calculation started (calculation_id=0ec3e865-1ef3-8f2c-8d0e-cf9f9296ea5f) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [27]:
with tempfile.TemporaryFile(mode='w+t') as f:
    # write the DataFrame to the temporary file
    df_rfm_loyalty.to_csv(f, index=False)
    # reset the file pointer to the beginning of the file
    f.seek(0)
    # upload the file to S3
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('athenapyspark-rfm-analysis-result')
    bucket.put_object(Key='analysis-results/evo_loyalty_rfm.csv', Body=f.read().encode())

Calculation started (calculation_id=d6c3e865-246a-2702-3618-c92099f50c24) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
0
s3.Object(bucket_name='athenapyspark-rfm-analysis-result', key='analysis-results/evo_loyalty_rfm.csv')



In [28]:
# Restricting the dataset (Monetary < 100000 & Frequency < 500) to best fit the chart

df_rfm_loyalty = (rfm_loyalty
    .filter(F.col("Monetary") < 100000)
    .filter(F.col("Frequency") < 500)
    .filter(F.col("Recency") < 500)
).toPandas()

Calculation started (calculation_id=eec3e865-29d3-3433-0752-bf4fc5b8dee0) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [29]:
# Monetary vs Recency for each segment
plt.close('all')
#fig, axs = plt.subplots(3, 1, figsize=(8, 8))
#sns.lmplot( data=df_rfm_loyalty, x="Recency", y="Monetary", hue='Loyalty')
#fig.savefig('rfm_monetary_vs_recency_lmplot.png')
#s3client.upload_file('rfm_monetary_vs_recency_lmplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_monetary_vs_recency_lmplot.png')

# create a figure with 3 subplots


# plot lmplot on

# save the figure to a file
#fig.savefig('rfm_monetary_vs_recency_lmplot.png')
#s3client.upload_file('rfm_monetary_vs_recency_lmplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_monetary_vs_recency_lmplot.png')
# Monetary vs Requency (combined)
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
sns.scatterplot(x='Recency', y='Monetary', data=df_rfm_loyalty, hue='Loyalty', size='Monetary')
fig.savefig('rfm_monetary_vs_recency_scatterplot.png')
s3client.upload_file('rfm_monetary_vs_recency_scatterplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_monetary_vs_recency_scatterplot.png')

# Frequency vs Recency for each segment
#sns.lmplot(x='Recency', y='Frequency', data=df_rfm_loyalty, col='Loyalty', hue='Loyalty', fit_reg=True)
#fig.savefig('rfm_frequency_vs_recency_lmplot.png')
#s3client.upload_file('rfm_frequency_vs_recency_lmplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_frequency_vs_recency_lmplot.png')

# Frequency vs Recency (combined)
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
sns.scatterplot(x='Recency', y='Frequency', data=df_rfm_loyalty, hue='Loyalty', size='Monetary')
fig.savefig('rfm_frequency_vs_recency_scatterplot.png')
s3client.upload_file('rfm_frequency_vs_recency_scatterplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_frequency_vs_recency_scatterplot.png')

# Monetary vs Frequency for each segment
#sns.lmplot(x='Monetary', y='Frequency', data=df_rfm_loyalty, col='Loyalty', hue='Loyalty', fit_reg=True)
#fig.savefig('rfm_monetary_vs_frequency_lmplot.png')
#s3client.upload_file('rfm_monetary_vs_frequency_lmplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_monetary_vs_frequency_lmplot.png')

# Monetary vs Frequency (combined)
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
sns.scatterplot(x='Monetary', y='Frequency', data=df_rfm_loyalty, hue='Loyalty', size='Monetary')
fig.savefig('rfm_monetary_vs_frequency_scatterplot.png')
s3client.upload_file('rfm_monetary_vs_frequency_scatterplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_monetary_vs_frequency_scatterplot.png')

Calculation started (calculation_id=d6c3e865-3b53-1b10-d8a9-c439f8a8a8ed) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Segmentation 2

In [30]:
# Assign loyalty level to each customer

# Champion: Loyal Customers, they are the champions with highest score of RFM.
# Potential: Customers who have the most potential to become the Champions.
#            They recently shopped and their purchases frequency is high as well. 
#            Moreover, the amount they spent is quite good.
# Require Attention: Customers who used to shop a lot with high money. but they did not purchase recently. 
# Lost: Customers who haven't paid a visit to the store for a long time.

segments = {
    'Champion':['111'], 
    'Potential':['112','113','121','131','123','122','133','132'], 
    'Need Attention':['211','221','231','232','213','212','233','223','222'],
    'Lost':['312','321','331','311','333','332','323','322','313']
}

rfm_segment = (rfm_agg_scores
    .withColumn("Segment",
                F.when((F.col("RFM_ScoreGroup").isin(segments['Champion'])), F.lit('Champion'))
                 .when((F.col("RFM_ScoreGroup").isin(segments['Potential'])), F.lit('Potential'))
                 .when((F.col("RFM_ScoreGroup").isin(segments['Need Attention'])), F.lit('Need Attention'))
                 .otherwise(F.lit('Lost'))))

Calculation started (calculation_id=78c3e865-40b8-714d-5d99-f85edea83aaf) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [31]:
# printdf(rfm_loyalty.filter(F.col("CustomerID").isin([12346,12747,12748,12749,12820])))
# printdf(rfm_loyalty.filter(F.col("RFM_ScoreGroup") == "111"), 10)
printdf(rfm_segment)

Calculation started (calculation_id=cac3e865-460a-dd9e-50a7-d6f729f8feb1) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
   user_id  Recency  Frequency  ...  RFM_Score  RFM_ScoreGroup         Segment
0        5      181         17  ...          4             211  Need Attention
1       12       72         14  ...          4             112       Potential
2       24      170          8  ...          4             211  Need Attention
3       31      192          6  ...          5             212  Need Attention
4       34      234         16  ...          5             212  Need Attention

[5 rows x 10 columns]



In [32]:
df_rfm_segment = rfm_segment.toPandas().sort_values(["RFM_ScoreGroup"])

fig, ax = plt.subplots(1, 1, figsize=(6, 8))

# sns.histplot(df_rfm_loyalty["Loyalty"], color='steelblue', shrink=.8)
sns.countplot(x="Segment", data=df_rfm_segment, color='steelblue')
for p in ax.patches:
    ax.annotate('{}'.format(p.get_height()), (p.get_x() + 0.3, p.get_height() + 1))
fig.savefig('rfm_segment_countplot.png')
import boto3
s3client=boto3.client('s3')
s3client.upload_file('rfm_segment_countplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_segment_countplot.png')

Calculation started (calculation_id=b6c3e865-5789-9f35-7ddf-265698a0aeb8) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [33]:
with tempfile.TemporaryFile(mode='w+t') as f:
    df_rfm_segment.to_csv(f, index=False)
# reset the file pointer to the beginning of the file
    f.seek(0)
# upload the file to S3
    bucket.put_object(Key='analysis-results/evo_rfm_segment.csv', Body=f.read().encode())

Calculation started (calculation_id=84c3e865-5d05-864e-3062-c91f61b96242) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
0
s3.Object(bucket_name='athenapyspark-rfm-analysis-result', key='analysis-results/evo_rfm_segment.csv')



# Visualize

In [34]:
# Restricting the dataset (Monetary < 100000 & Frequency < 500) to optimize the chart

df_rfm_segment = (rfm_segment
    .filter(F.col("Monetary") < 100000)
    .filter(F.col("Frequency") < 500)
    .filter(F.col("Recency") < 500)
).toPandas()

Calculation started (calculation_id=36c3e865-6267-d53d-f989-c8609d2aa238) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [35]:
# Monetary vs Frequency for each segment
#sns.lmplot(x='Monetary', y='Frequency', data=df_rfm_segment, col='Segment', hue='Segment', fit_reg=True)

# Monetary vs Frequency (combined)
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
sns.scatterplot(x='Monetary', y='Frequency', data=df_rfm_segment, hue='Segment', size='Monetary')
fig.savefig('rfm_visualize_monetary_vs_frequency_scatterplot.png')
s3client.upload_file('rfm_visualize_monetary_vs_frequency_scatterplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_visualize_monetary_vs_frequency_scatterplot.png')

Calculation started (calculation_id=90c3e865-7830-4832-4070-6e670258ea87) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [36]:
import matplotlib
matplotlib.rcParams['figure.max_open_warning'] = 50 # set the maximum number of allowed open figures to 50

Calculation started (calculation_id=54c3e865-7d9b-8411-e5b0-1b09f2dd43ed) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [37]:
# Monetary vs Recency for each segment
#sns.lmplot(x='Recency', y='Monetary', data=df_rfm_segment, col='Segment', hue='Segment', fit_reg=True)

# Monetary vs Recency (combined)
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
sns.scatterplot(x='Recency', y='Monetary', data=df_rfm_segment, hue='Segment', size='Monetary')
fig.savefig('rfm_visualize_monetary_vs_recency_scatterplot.png')
s3client.upload_file('rfm_visualize_monetary_vs_recency_scatterplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_visualize_monetary_vs_recency_scatterplot.png')

Calculation started (calculation_id=38c3e865-7ec9-fb92-7597-af3a8b2869d1) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [38]:
# Frequency vs Recency for each segment
#sns.lmplot(x='Recency', y='Frequency', data=df_rfm_segment, col='Segment', hue='Segment', fit_reg=True)

# Frequency vs Recency (combined)
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
sns.scatterplot(x='Recency', y='Frequency', data=df_rfm_segment, hue='Segment', size='Monetary')
fig.savefig('rfm_visualize_frequency_vs_recency_scatterplot.png')
s3client.upload_file('rfm_visualize_frequency_vs_recency_scatterplot.png', 'athenapyspark-rfm-analysis-result', 'analysis-results/rfm_visualize_frequency_vs_recency_scatterplot.png')

Calculation started (calculation_id=acc3e865-840e-894d-e23c-6d705b7008a3) in (session=82c3e85f-07fe-ef66-49a7-42c131c664cf). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [1]:
# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(projection='3d')

# colors = {'Champion':'green', 'Potential':'blue', 'Need Attention':'orange', 'Lost':'red'}

# ax.scatter(xs=df_rfm_segment.Frequency, 
#            ys=df_rfm_segment.Recency, 
#            zs=df_rfm_segment.Monetary, 
#            c=[colors[x] for x in df_rfm_segment.Segment])